## Add control values

In [1]:
if '../' not in sys.path:
    sys.path.append('../')
from src.data.parse_data import *
from src.visualization.advanced_visualizations import *

In [2]:
df_list = list()
for i in range(1,8):
    df_list.append((parse_data(f'Assay {i}.xlsx'), f'Assay {i}.xlsx'))

Assay 1.xlsx - deleted 15 rows with invalid Transfer Status
Assay 2.xlsx - deleted 4 rows with invalid Transfer Status
Assay 3.xlsx - deleted 2 rows with invalid Transfer Status


In [3]:
combined = combine_assays(df_list)
# combined.head()

In [4]:
df_prepared = get_activation_inhibition(combined)
# df_prepared.head()

In [5]:
res = add_control_rows(df_prepared)
# res.tail()

In [6]:
projections = get_projections(res)
# projections.tail()

In [7]:
compounds, ctrl = split_compounds_controls(projections)
# display(compounds.head())
# display(ctrl.head(2))

In [73]:
def projection_2d_add_controls2(fig: px.scatter, controls: dict[pd.DataFrame], projection: str = 'umap') -> px.scatter:
    """
    Add control values to the plot of selected projection.
    
    :param fig: projection plot

    :param controls: dictionary containing data frames with projected control values (positive and negative respectively)
    
    :param projection: name of projection to be visualized

    """
    fig_controls = go.Figure(fig)
    fig_controls.update_traces(marker={"opacity": 0.6})

    control_styles = {
        'all_neg': ['#de425b',12],
        'all_but_one_neg': ['#eb7a52',10], 
        'neg': ['#f8b267',8],
        'all_pos': ['#488f31',12],
        'all_but_one_pos': ['#8aac49',10],
        'pos': ['#c6c96a',8]
    }
    for key in control_styles.keys():
        fig_controls.add_scatter(
                    x=controls[key][f'{str.upper(projection)}_X'],
                    y=controls[key][f'{str.upper(projection)}_Y'], 
                    mode='markers',
                    marker=dict(size=control_styles[key][1], color=control_styles[key][0], symbol='star-diamond'),
                    name= str.upper(key).replace('_',' '),
                    text = controls[key]['CMPD ID'].str.split(';'),
                    hovertemplate="<b>%{text[0]}</b><br>" +
                    "<b>%{text[1]}</b><br>" +
                    "X: %{x:.4f}<br>Y: %{y:.4f}<br>"+
                    "<extra></extra>")
    # OPTION 1
    fig_controls.update_layout(
        legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.15,
        xanchor="left",
        x=0
        ))

    # OPTION 2
    # fig_controls.update_layout(
    #     legend=dict(
    #     title="CONTROLS",
    #     yanchor="bottom",
    #     y=0.01,
    #     xanchor="left",
    #     x=0.01,
    #     font=dict(size=10)),
    #     title_x=0.5)
    return fig_controls

In [74]:
fig = plot_projection_2d(compounds, '% ACTIVATION - Assay 2')
controls = split_controls_pos_neg(ctrl, '% ACTIVATION - Assay 2')
fig_controls = projection_2d_add_controls2(fig, controls)
fig_controls.show()
fig.show()